In [1]:
from transformers import LEDForConditionalGeneration, AutoConfig, AutoTokenizer, BigBirdPegasusForConditionalGeneration, PegasusForConditionalGeneration
import torch
import pandas as pd
import numpy as np
import wandb
from pathlib import Path

In [2]:
dummy_input = 'Device and Usage Requirements. Certain types of Paid Services may require you to use a device that meets the system and compatibility requirements for such Paid Service, which may change from time to time.'

In [2]:
text = '''4. Your license to use the products
Blackboard gives you a personal, worldwide, royalty-free, non-assignable, and non-exclusive license to use the Products, subject to these Terms and any agreement entered into by your institution relating to the Products, solely for the intended use of the Products, and not for competitive intelligence, analysis, or demonstrations. You may not copy, modify, distribute, sell, or lease any part of our Products, nor may you reverse engineer or attempt to extract the source code of any Product, unless laws prohibit those restrictions or you have our written permission.

5. Mobile software
A. Mobile Software

We may make available software to access the Products via a mobile device (“Mobile Software”). To use the Mobile Software you must have a mobile device that is compatible with the Mobile Software. Blackboard does not warrant that the Mobile Software will be compatible with your mobile device. You may use mobile data in connection with the Mobile Software and may incur additional charges from your wireless provider for these services. You agree that you are solely responsible for any such charges. Blackboard hereby grants you a non-exclusive, non-transferable, revocable license to use a compiled code copy of the Mobile Software for one account on one mobile device owned or leased solely by you, for your personal use. You may not: (i) modify, disassemble, decompile or reverse engineer the Mobile Software, except to the extent that such restriction is expressly prohibited by law; (ii) rent, lease, loan, resell, sublicense, distribute or otherwise transfer the Mobile Software to any third party or use the Mobile Software to provide time sharing or similar services for any third party; (iii) make any copies of the Mobile Software; (iv) remove, circumvent, disable, damage or otherwise interfere with security-related features of the Mobile Software, features that prevent or restrict use or copying of any content accessible through the Mobile Software, or features that enforce limitations on use of the Mobile Software; or (v) delete the copyright and other proprietary rights notices on the Mobile Software. You acknowledge that Blackboard may from time to time issue upgraded versions of the Mobile Software, and may automatically electronically upgrade the version of the Mobile Software that you are using on your mobile device. You consent to such automatic upgrading on your mobile device, and agree that the terms and conditions of this Agreement will apply to all such upgrades. Any third-party code that may be incorporated in the Mobile Software is covered by the applicable open source or third-party license EULA, if any, authorizing use of such code. The foregoing license grant is not a sale of the Mobile Software or any copy thereof, and Blackboard or its third-party partners or suppliers retain all right, title, and interest in the Mobile Software (and any copy thereof). Any attempt by you to transfer any of the rights, duties or obligations hereunder, except as expressly provided for in this Agreement, is void. Blackboard reserves all rights not expressly granted under these Terms. The Mobile Software originates in the United States, and is subject to United States export laws and regulations. The Mobile Software may not be exported or re-exported to certain countries or those persons or entities prohibited from receiving exports from the United States. In addition, the Mobile Software may be subject to the import and export laws of other countries. You agree to comply with all United States and foreign laws related to use of the Mobile Software and the Products.

B. Mobile Software from App Store by Apple

The following applies to any Mobile Software you acquire from the App Store (“App Store-Sourced Software”): You acknowledge and agree that these Terms are solely between you and Blackboard, not Apple, and that Apple has no responsibility for the App Store-Sourced Software or content thereof. Your use of the App Store-Sourced Software must comply with the App Store Terms of Service. You acknowledge that Apple has no obligation whatsoever to furnish any maintenance and support services with respect to the App Store-Sourced Software. In the event of any failure of the App Store-Sourced Software to conform to any applicable warranty, you may notify Apple, and Apple will refund the purchase price for the App Store-Sourced Software to you; to the maximum extent permitted by applicable law, Apple will have no other warranty obligation whatsoever with respect to the App Store-Sourced Software, and any other claims, losses, liabilities, damages, costs or expenses attributable to any failure to conform to any warranty will be solely governed by these Terms and any law applicable to Blackboard as provider of the software. You acknowledge that Apple is not responsible for addressing any of your claims or those of any third party relating to the App Store-Sourced Software or your possession and/or use of the App Store-Sourced Software, including, but not limited to: (i) product liability claims; (ii) any claim that the App Store-Sourced Software fails to conform to any applicable legal or regulatory requirement; and (iii) claims arising under consumer protection or similar legislation; and all such claims are governed solely by this Agreement and any law applicable to Blackboard as provider of the software. You acknowledge that, in the event of any third-party claim that the App Store-Sourced Software or your possession and use of that App Store-Sourced Software infringes that third party’s intellectual property rights, Blackboard, not Apple, will be solely responsible for the investigation, defense, settlement and discharge of any such intellectual property infringement claim to the extent required by these Terms. You and Blackboard acknowledge and agree that Apple, and Apple’s subsidiaries, are third-party beneficiaries of these Terms as relates to your license of the App Store-Sourced Software, and that, upon your acceptance of the Terms, Apple will have the right (and will be deemed to have accepted the right) to enforce these Terms as relates to your license of the App Store-Sourced Software against you as a third-party beneficiary thereof.

C. Mobile Software from Google Play Store.

The following applies to any Mobile Software you acquire from the Google Play Store (“Google-Sourced Software”): (i) you acknowledge that the Terms are between you and Blackboard only, and not with Google, Inc. (“Google”); (ii) your use of Google-Sourced Software must comply with Google’s then-current Google Play Store Terms of Service; (iii) Google is only a provider of the Google Play Store where you obtained the Google-Sourced Software; (iv) Blackboard, and not Google, is solely responsible for its Google-Sourced Software; (v) Google has no obligation or liability to you with respect to Google-Sourced Software or the Terms; and (vi) you acknowledge and agree that Google is a third-party beneficiary to the Terms as relates to Blackboard’s Google-Sourced Software.

D. Mobile Software from Microsoft Store

The following applies to any Mobile Software you acquire from the Microsoft Store ("MS-Sourced Software"): (i) you acknowledge that the Agreement is between you and Blackboard only, and not with Microsoft Corporation ("Microsoft"); (ii) your use of MS-Sourced Software must comply with Microsoft's then-current Microsoft Store Terms of Service; (iii) Microsoft is only a provider of the Microsoft Store where you obtained the MS-Sourced Software; (iv) Blackboard, and not Microsoft, is solely responsible for its MS-Sourced Software; (v) Microsoft has no obligation or liability to you with respect to MS-Sourced Software or this Agreement; and (vi) you acknowledge and agree that Microsoft is a third-party beneficiary to the Agreement as it relates to Blackboard’s MS-Sourced Software.

6. Blackboard rights
The Products are always evolving and the form, nature, and/or functionality of the Products may change from time to time without prior notice to you. In addition, Blackboard may stop (permanently or temporarily) providing the Products (or any features within the Products) to you or to users generally and may not be able to provide you with prior notice. We also retain the right to create limits on use and storage at our sole discretion at any time without prior notice to you.

All right, title, and interest in and to the Products (excluding Content provided by users or other third parties) are and will remain the exclusive property of Blackboard and its licensors. The Products are protected by copyright, trademark, and other laws of both the United States and foreign countries. Nothing in the Terms gives you a right to use the Blackboard name or any of the Blackboard trademarks, logos, domain names, and other distinctive brand features. Any feedback, comments, or suggestions you may provide regarding Blackboard, or the Products is entirely voluntary and we will be free to use such feedback, comments or suggestions as we see fit and without any obligation to you.

7. Registration obligations & passwords
You may never use another User’s account without permission. In consideration of your use of the Products, you agree (a) that Your Information will be true, accurate, current and complete, and (b) to maintain and promptly update Your Information to keep it true, accurate, current and complete. You are responsible for safeguarding any password that you use to access the Products (including passwords used on other Blackboard products that permit single sign-on to the Products) and for any activities or actions under your password. We encourage you to use “strong” passwords (passwords that use a combination of upper and lower case letters, numbers and symbols) with your account. You agree to (a) notify Blackboard of any unauthorized use of your password or user identification and any other breach of security, and (b) ensure that you exit from your account at the end of each session.

8. Export control
As required by the laws of the United States and other countries: (a) you understand that the Products are subject to export controls under the U.S. Commerce Department's Export Administration Regulations ("EAR"); (b) you are not located in a prohibited destination country under the EAR or U.S. sanctions regulations; and (c) you will not export, re-export, or transfer the Products to any prohibited destination or persons or entities on the U.S. Bureau of Industry and Security Denied Parties List or Entity List, or the U.S. Office of Foreign Assets Control list of Specially Designated Nationals and Blocked Persons, or any similar lists maintained by other countries, without the necessary export license(s) or authorization(s).

9. Termination
We may suspend or terminate your accounts or cease providing you with all or part of the Products at any time for any reason, including, but not limited to, if we reasonably believe: (i) you have violated these Terms, (ii) you create risk or possible legal exposure for us; or (iii) our provision of the Products is no longer commercially viable. In such a termination event, the license granted hereunder shall automatically terminate. In all such cases, the Terms shall terminate, including, without limitation, your license to use the Products, except that the following Sections shall continue to apply: 2, 3, 5, 6, 9, 10, and 11.

Further, if you stop using the Products for more than 60 days, your accounts may be deleted and Blackboard assumes no responsibility for Content deleted following inactivity. If you wish to delete your Blackboard Product account(s) in their entirety(ies), you may contact us via GeneralCounsel@blackboard.com and specify which Blackboard Product account(s) you wish to delete. We will delete your specified accounts within a reasonable time, unless prohibited by law.

Nothing in this Section shall affect Blackboard’s rights to change, limit, or stop the provision of the Products without prior notice, as provided above in Section 5.

10. Disclaimers & limitations of liability
Please read this Section carefully since it limits the liability of Blackboard and its parents, subsidiaries, affiliates, related companies, officers, directors, employees, agents, representatives, partners, and/or licensors (collectively, the “Blackboard Entities”). Each of the Subsections below only applies up to the maximum extent permitted under applicable law. Nothing in this Section is intended to limit any rights you may have which may not be lawfully limited.

A. The Products are available “AS-IS”

Your access to and use of the Products or any Content are at your own risk. You understand and agree that the Products are provided to you on an “AS IS” and “AS AVAILABLE” basis. Without limiting the foregoing, to the maximum extent permitted under applicable law, THE BLACKBOARD ENTITIES DISCLAIM ALL WARRANTIES AND CONDITIONS, WHETHER EXPRESS OR IMPLIED, OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR NON-INFRINGEMENT. No advice or information, whether oral or written, obtained from any of the Blackboard Entities or through the Products, will create any warranty not expressly made herein.

B. HIPAA Disclaimer

Blackboard does not warrant or provide any assurances that your use of the Products will comply with the Health Insurance Portability and Accountability Act of 1996, as amended (“HIPAA”). If you are a health care provider, health plan, insurer, health care information clearinghouse or other entity that may be considered a “covered entity” under HIPAA or a “business associate” of a “covered entity” under HIPAA, you shall take all steps necessary to comply with HIPAA and shall be responsible for any applicable compliance with HIPAA. You agree, that you will not provide Blackboard with any protected health information (as that term is defined under HIPAA) and that Blackboard is therefore not deemed to be a business associate of Client for purposes of compliance with HIPAA.

C. Links

The Products may contain links to third-party websites or resources. You acknowledge and agree that the Blackboard Entities are not responsible or liable for: (i) the availability or accuracy of such websites or resources; or (ii) the content, products, or services on or available from such websites or resources. Links to such websites or resources do not imply any endorsement by the Blackboard Entities of such websites or resources or the content, products, or services Available from such websites or resources. You acknowledge sole responsibility for and assume all risk arising from your use of any such websites or resources.

D. Limitation of liability

TO THE MAXIMUM EXTENT PERMITTED BY APPLICABLE LAW, THE BLACKBOARD ENTITIES SHALL NOT BE LIABLE FOR ANY INDIRECT, INCIDENTAL, SPECIAL, CONSEQUENTIAL OR PUNITIVE DAMAGES, OR ANY LOSS OF PROFITS OR REVENUES, WHETHER INCURRED DIRECTLY OR INDIRECTLY, OR ANY LOSS OF DATA, USE, GOOD-WILL, OR OTHER INTANGIBLE LOSSES, RESULTING FROM (i) YOUR ACCESS TO OR USE OF OR INABILITY TO ACCESS OR USE THE PRODUCTS; (ii) ANY CONDUCT OR CONTENT OF ANY THIRD PARTY ON,THROUGH, OR ASSOCIATED WITH THE PRODUCTS, INCLUDING WITHOUT LIMITATION, ANY DEFAMATORY, OFFENSIVE OR ILLEGAL CONDUCT OF OTHER USERS OR THIRD PARTIES; (iii) ANY CONTENT OBTAINED FROM THE PRODUCTS; OR (iv) UNAUTHORIZED ACCESS, USE OR ALTERATION OF YOUR TRANSMISSIONS OR CONTENT.

IN NO EVENT SHALL THE AGGREGATE LIABILITY OF THE BLACKBOARD ENTITIES EXCEED THE GREATER OF ONE HUNDRED U.S. DOLLARS (U.S. $100.00) OR THE AMOUNT YOU PAID BLACKBOARD, IF ANY, IN THE PAST SIX MONTHS FOR THE PRODUCTS GIVING RISE TO THE CLAIM.

THE LIMITATIONS OF THIS SUBSECTION SHALL APPLY TO ANY THEORY OF LIABILITY, WHETHER BASED ON WARRANTY, CONTRACT, STATUTE, TORT (INCLUDING NEGLIGENCE) OR OTHERWISE, AND WHETHER OR NOT THE BLACKBOARD ENTITIES HAVE BEEN INFORMED OF THE POSSIBILITY OF ANY SUCH DAMAGE, AND EVEN IF A REMEDY SET FORTH HEREIN IS FOUND TO HAVE FAILED OF ITS ESSENTIAL PURPOSE.

11. General terms
A. Waiver & severability

The failure of Blackboard to enforce any right or provision of these Terms will not be deemed a waiver of such right or provision. In the event that any provision of these Terms is held to be invalid or unenforceable, then that provision will be limited or eliminated to the minimum extent necessary, and the remaining provisions of these Terms will remain in full force and effect.

B. Governing law

These Terms and any action related thereto will be governed by the laws of the State of New York without regard to or application of its conflict of law provisions or your state or country of residence. All claims, legal proceedings, or litigation arising in connection with the Products will be brought solely in the federal or state courts located in Washington, D.C., United States, and you consent to the jurisdiction of and venue in such courts and waive any objection as to inconvenient forum.

C. U.S. Government users & U.S. Government restricted rights

Certain of the components that comprise the Products are “commercial items” as that term is defined at 48 C.F.R. 2.101, consisting of “commercial computer software” and/or “commercial computer software documentation” as such terms are used in 48 C.F.R. 12.212. Consistent with 48 C.F.R. 12.212 and 48 C.F.R. 227.7202-1 through 227.7202-4, all U.S. Government end users acquire the Software with only those rights set forth herein.

The Products, any Mobile Software and all upgrades to the foregoing (if any) are provided with restricted rights. Use, duplication, or disclosure by the U.S. Government is subject to restrictions as set forth in subparagraph (c)(1)(ii) of the Rights in Technical Data and Computer Software clause at DFARS 252.227-7013 or subparagraphs (c)(1) and (2) of the Commercial Computer Software-Restricted Rights at 48 CFR 52.227-19, as applicable.

Contractor/manufacturer is Blackboard Inc., 1111 19th Street N.W., Washington, D.C. 20006. All rights not specifically granted in this Agreement are reserved by Blackboard.

D. Entire agreement

These Terms and our Privacy Policy are the entire and exclusive agreement between Blackboard and you regarding the Products, and these Terms supersede and replace any prior agreements between Blackboard and you regarding the Products. Other than members of the group of companies of which Blackboard Inc. is the parent, no other person or company will be third party beneficiaries to the Terms.

Notwithstanding the foregoing, nothing in these Terms supersedes or limits your rights under (1) the terms and conditions of any written agreement you have entered into with Blackboard regarding the use of Products, or (2) applicable laws or regulations to the extent these Terms are prohibited by such laws or regulations. In the event of any conflict between these Terms and the terms and conditions of an applicable written agreement you have entered into with Blackboard, the terms and conditions of the written agreement shall control.

We may revise these Terms from time-to-time, the most current version will always be at the referenced link. If the revision, in our sole discretion, is material we will notify you through the Products or through the email associated with your profile. By continuing to access or use the Products after those revisions become effective, you agree to be bound by the revised Terms.

These Products are operated and provided by Blackboard:

'''

In [11]:
def summarize(text, tokenizer):
    input_tokenized = tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=800,
                                  max_length=1000,
                                  early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

    return summary

In [ ]:
def measure_model_performance(model, tokenizer):
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions=10
    timings = np.zeros((repetitions, 1))

    # GPU-WARM-UP
    for _ in range(5):
        _ = summarize(dummy_input, tokenizer)

    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):

            starter.record()
#             _ = df["original_text"].apply(lambda x: summarize(x))
            _ = summarize(text, tokenizer)
            ender.record()

            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time

    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    
    return mean_syn, std_syn

In [4]:
run = wandb.init(project = "major-one", name = "Testing 6000")
artifact = run.use_artifact("model-bigbird:v1")
datadir = artifact.download()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact model-bigbird:v1, 2209.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:29.8


In [5]:
config_path = datadir + "/config.json"
model_path = datadir + "/pytorch_model.bin"
vocab_path = datadir + "/vocab.json"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
config = AutoConfig.from_pretrained(config_path)
model = BigBirdPegasusForConditionalGeneration(config)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
BigBird_tokenizer = AutoTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')

In [8]:
model.to(device)

BigBirdPegasusForConditionalGeneration(
  (model): BigBirdPegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): BigBirdPegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): BigBirdPegasusLearnedPositionalEmbedding(4096, 1024)
      (layers): ModuleList(
        (0-15): 16 x BigBirdPegasusEncoderLayer(
          (self_attn): BigBirdPegasusEncoderAttention(
            (self): BigBirdPegasusBlockSparseAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=False)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=False)
            )
            (output): Linear(in_features=1024, out_features=1024, bias=False)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=1

In [ ]:
mean_time, std_time = measure_model_performance(model, BigBird_tokenizer)
print(f"Mean Inference Time: {mean_time/1000} s, Std Dev: {std_time/1000}")

In [9]:
test = ''' Introduction
Thank you for using the YouTube products, services and features provided to you from the platform (collectively, the “Service”).
Our Service
The Service allows you to discover, watch and share videos and other content, provides a forum for people to connect, inform, and inspire others across the globe, and acts as a distribution platform for original content creators and advertisers large and small. We provide lots of information about our products and how to use them in our Help Center. Among other things, you can find out about YouTube Kids, the YouTube Partner Program and YouTube Paid Memberships and Purchases (where available).You can also read all about enjoying content on other devices like your television, your games console, or Google Home.
Your Service Provider
The entity providing the Service is Google LLC, a company operating under the laws of Delaware, located at 1600 Amphitheatre Parkway, Mountain View, CA 94043 (referred to as “YouTube”, “we”, “us”, or “our”). References to YouTube’s “Affiliates” in these terms means the other companies within the Alphabet Inc. corporate group (now or in the future).
Applicable Terms
Your use of the Service is subject to these terms, the YouTube Community Guidelines and the Policy, Safety and Copyright Policies which may be updated from time to time (together, this "Agreement"). Your Agreement with us will also include the Advertising on YouTube Policies if you provide advertising or sponsorships to the Service or incorporate paid promotions in your content. Any other links or references provided in these terms are for informational use only and are not part of the Agreement.
Please read this Agreement carefully and make sure you understand it. If you do not understand the Agreement, or do not accept any part of it, then you may not use the Service.
By accepting this Agreement, you affirm that you are 18 years of age or above and are fully competent to enter into this Agreement, and to abide by and comply with this Agreement. In case you are below 18 years of age, the restrictions and requirements below apply.
Who may use the Service?
Age Requirements
You must be at least 13 years old to use the Service; however, children of all ages may use the Service and YouTube Kids (where available) if enabled by a parent or legal guardian.
Permission by Parent or Guardian
If you are considered a minor in your country, you represent that you have your parent or guardian’s permission to use the Service. Please have them read this Agreement with you.
If you are a parent or legal guardian of a minor in your country, by allowing your child to use the Service, you are subject to the terms of this Agreement and responsible for your child’s activity on the Service. You can find tools and resources to help you manage your family’s experience on YouTube (including how to enable a child under the age of 13 to use the Service and YouTube Kids) in our Help Center and through Google’s Family Link.
Businesses
If you are using the Service on behalf of a company or organisation, you represent that you have authority to act on behalf of that entity, and that such entity accepts this Agreement.
Your Use of the Service
Content on the Service
The content on the Service includes videos, audio (for example music and other sounds), graphics, photos, text (such as comments and scripts), branding (including trade names, trademarks, service marks, or logos), interactive features, software, metrics, and other materials (collectively, "Content”).
Content is the responsibility of the person or entity that provides it to the Service. YouTube is under no obligation to host or serve Content. If you see any Content you believe does not comply with this Agreement, including by violating the Community Guidelines or the law, you can report it to us.
Google Accounts and YouTube Channels
You can use parts of the Service, such as browsing and searching for Content, without having a Google account. However, you do need a Google account to use some features. With a Google account, you may be able to like videos, subscribe to channels, create your own YouTube channel, and more. You can follow these instructions to create a Google account.
Creating a YouTube channel will give you access to additional features and functions, such as uploading videos, making comments or creating playlists (where available). Here are some details about how to create your own YouTube channel.
To protect your Google account, keep your password confidential. You should not reuse your Google account password on third-party applications. Learn more about keeping your Google account secure, including what to do if you learn of any unauthorised use of your password or Google account.
Your Information
Our Privacy Policy explains how we treat your personal data and protect your privacy when you use the Service. The YouTube Kids Privacy Notice provides additional information about our privacy practices that are specific to YouTube Kids.
We will process any audio or audiovisual content uploaded by you to the Service in accordance with the YouTube Data Processing  Terms, except in cases where you uploaded such content for personal purposes or household activities. Learn More.
Permissions and Restrictions
You may access and use the Service as made available to you, as long as you comply with this Agreement and applicable law. You may view or listen to Content for your personal, non-commercial use. You may also show YouTube videos through the embeddable YouTube player.
The following restrictions apply to your use of the Service. You are not allowed to:
access, reproduce, download, distribute, transmit, broadcast, display, sell, license, alter, modify or otherwise use any part of the Service or any Content except: (a) as expressly authorized by the Service; or (b) with prior written permission from YouTube and, if applicable, the respective rights holders;
circumvent, disable, fraudulently engage with, or otherwise interfere with any part of the Service (or attempt to do any of these things), including security-related features or features that (a) prevent or restrict the copying or other use of Content or (b) limit the use of the Service or Content;
access the Service using any automated means (such as robots, botnets or scrapers) except (a) in the case of public search engines, in accordance with YouTube’s robots.txt file; or (b) with YouTube’s prior written permission;
collect or harvest any information that might identify a person (for example, usernames or faces), unless permitted by that person or allowed under section (3) above;
use the Service to distribute unsolicited promotional or commercial content or other unwanted or mass solicitations;
cause or encourage any inaccurate measurements of genuine user engagement with the Service, including by paying people or providing them with incentives to increase a video’s views, likes, or dislikes, or to increase a channel’s subscribers, or otherwise manipulate metrics in any manner;
misuse any reporting, flagging, complaint, dispute, or appeals process, including by making groundless, vexatious, or frivolous submissions;
run contests on or through the Service that do not comply with YouTube’s contest policies and guidelines;
use the Service to view or listen to Content other than for personal, non-commercial use
(for example, you may not publicly screen videos or stream music from the Service); or
use the Service to (a) sell any advertising, sponsorships, or promotions placed on, around, or within the Service or Content, other than those allowed in the Advertising on YouTube policies (such as compliant product placements); or (b) sell advertising, sponsorships, or promotions on any page of any website or application that only contains Content from the Service or where Content from the Service is the primary basis for such sales (for example, selling ads on a webpage where YouTube videos are the main draw for users visiting the webpage).
Reservation
Using the Service does not give you ownership of or rights to any aspect of the Service, including user names or any other Content posted by others or YouTube).
Develop, Improve and Update the Service
YouTube is constantly changing and improving the Service. As part of this continual evolution, we may make modifications or changes (to all or part of the Service) such as adding or removing features and functionalities, offering new digital content or services or discontinuing old ones. We may also need to alter or discontinue the Service, or any part of it, in order to make performance or security improvements, make changes to comply with law, or prevent illegal activities on or abuse of our systems. These changes may affect all users, some users or even an individual user. When the Service requires or includes downloadable software (such as the YouTube Studio application), that software may update automatically on your device once a new version or feature is available, subject to your device settings. If we make material changes that negatively impact your use of the Service, we’ll provide you with reasonable advance notice, except in urgent situations such as preventing abuse, responding to legal requirements, or addressing security and operability issues. We’ll also provide you with an opportunity to export your content from your Google Account using Google Takeout, subject to applicable law and policies.
Your Content and Conduct
Uploading Content
If you have a YouTube channel, you may be able to upload Content to the Service. You may use your Content to promote your business or artistic enterprise. If you choose to upload Content, you must not submit to the Service any Content that does not comply with this Agreement (including the YouTube Community Guidelines) or the applicable law(s).
In particular, the Content must:
a. respect the rights of others, including privacy;
b. not include third-party intellectual property (such as copyrighted material) unless you have permission from that party or are otherwise legally entitled to do so;
c. not abuse or harm others or yourself (or threaten or encourage such abuse or harm), including against children;
d. not mislead, be patently false, or defrauding;
e. not illegally impersonate, defame, bully, harass, be obscene or stalk others;
f. not incite violation of applicable laws;
g. not abuse, harm, interfere with, or disrupt the services — for example, by accessing or using them in fraudulent or deceptive ways, introducing malware, or spamming, hacking, or bypassing our systems or protective measures.
You are legally responsible for the Content you submit to the Service. We may use automated systems that analyze your Content to help detect infringement and abuse, including spam and malware.
Rights you Grant
You retain ownership rights in your Content. However, we do require you to grant certain rights to YouTube and other users of the Service, as described below.
License to YouTube
By providing Content to the Service, you grant to YouTube a worldwide, non-exclusive, royalty-free, transferable, sublicensable license to use that Content (including to reproduce, distribute, prepare derivative works, display and perform it). YouTube may only use that Content in connection with the Service and YouTube’s (and its successors’ and Affiliates) business, including for the purpose of promoting and redistributing part or all of the Service.
License to Other Users
You also grant each other user of the Service a worldwide, non-exclusive, royalty-free license to access your Content through the Service, and to use that Content, including to reproduce, distribute, prepare derivative works, display and perform it, only as enabled by a feature of the Service (such as video playback or embeds). For clarity, this license does not grant any rights or permissions for a user to make use of your Content independent of the Service.
Duration of License
The licenses granted by you continue for a commercially reasonable period of time after you remove or delete your Content from the Service. You understand and agree, however, that YouTube may retain, but not display, distribute, or perform, server copies of your videos that have been removed or deleted.
Right to Monetize
You grant to YouTube the right to monetize your Content on the Service (and such monetization may include displaying ads on or within Content or charging users a fee for access). This Agreement does not entitle you to any payments. Starting June 1, 2021, any payments you may be entitled to receive from YouTube under any other agreement between you and YouTube (including for example payments under the YouTube Partner Program, Channel memberships or Super Chat) will be treated as royalties.  If required by law, Google will withhold taxes from such payments.
Removing Your Content
You may remove your Content from the Service at any time. You also have the option to make a copy of your Content before removing it. You must remove your Content if you no longer have the rights required by these terms.
Removal of Content By YouTube
If we reasonably believe that any of your Content (1) is in breach of this Agreement or (2) may cause harm to YouTube, our users, or third parties, we reserve the right to remove or take down that Content in accordance with applicable law. We will notify you with the reason for our action unless we reasonably believe that to do so: (a) would breach the law or the direction of a legal enforcement authority or would otherwise risk legal liability for YouTube or our Affiliates; (b) would compromise an investigation or the integrity or operation of the Service; or (c) would cause harm to any user, other third party, YouTube or our Affiliates. You can learn more about reporting and enforcement, including how to appeal on the Troubleshooting page of our Help Center.
Community Guidelines Strikes
YouTube operates a system of “strikes” in respect of Content that violates the YouTube Community Guidelines. Each strike comes with varying restrictions and may result in the permanent removal of your channel from YouTube.  A full description of how a strike affects your channel is available on the Community Guidelines Strikes Basics page. If you believe that a strike has been issued in error, you may appeal here.
If your channel has been restricted due to a strike, you must not use another channel to circumvent these restrictions. Violation of this prohibition is a material breach of this Agreement and Google reserves the right to terminate your Google account or your access to all or part of the Service.
Copyright Protection
We provide information to help copyright holders manage their intellectual property online in our YouTube Copyright Center. If you believe your copyright has been infringed on the Service, please send us a notice.
We respond to notices of alleged copyright infringement according to the process in our YouTube Copyright Center, where you can also find information about how to resolve a copyright strike. YouTube's policies provide for the termination, in appropriate circumstances, of repeat infringers’ access to the Service.
Account Suspension & Termination
Terminations by You
You may stop using the Service at any time. Follow these instructions to delete the Service from your Google Account, which involves closing your YouTube channel and removing your data. You also have the option to download a copy of your data first.
Terminations and Suspensions by YouTube
YouTube reserves the right to suspend or terminate  your Google account or your access to all or part of the Service if: (a) you materially or repeatedly breach this Agreement; (b) we are required to do so to comply with a legal requirement or a court order; or (c) we believe there has been conduct that creates (or could create) liability or harm to any user, other third party, YouTube or our Affiliates.
Notice for Termination or Suspension
We will notify you with the reason for termination or suspension by YouTube unless we reasonably believe that to do so: (a) would violate the law or the direction of a legal enforcement authority; (b) would compromise an investigation; (c) would compromise the integrity, operation or security of the Service; or (d) would cause harm to any user, other third party, YouTube or our Affiliates. Where YouTube is terminating your use for Service changes, where reasonably possible, you will be provided with sufficient time to export your Content from the Service.
Effect of Account Suspension or Termination
If your Google account is terminated or your access to the Service is restricted, you may continue using certain aspects of the Service (such as viewing only) without an account, and this Agreement will continue to apply to such use. If you believe that the termination or suspension of your Google account has been made in error, you can appeal using this form.
About Software in the Service
Downloadable Software
When the Service requires or includes downloadable software (such as the YouTube Studio application), unless that software is governed by additional terms which provide a license, YouTube gives you a personal, worldwide, royalty-free, non-assignable and non-exclusive license to use the software provided to you by YouTube as part of the Service. This license is for the sole purpose of enabling you to use and enjoy the benefit of the Service as provided by YouTube, in the manner permitted by this Agreement. You are not allowed to copy, modify, distribute, sell, sublicense or lease any part of the software, unless you have YouTube’s written permission.
Open Source
Some software used in our Service may be offered under an open source, royalty-free license that we provide to you. There may be provisions in an open source license that expressly override some of these terms, so please be sure to read those licenses.
Other Legal Terms
Warranty Disclaimer
OTHER THAN AS EXPRESSLY STATED IN THIS AGREEMENT OR AS REQUIRED BY LAW, THE SERVICE IS PROVIDED “AS IS” AND YOUTUBE DOES NOT MAKE ANY SPECIFIC COMMITMENTS OR WARRANTIES ABOUT THE SERVICE. FOR EXAMPLE, WE DON’T MAKE ANY WARRANTIES ABOUT: (A) THE CONTENT PROVIDED THROUGH THE SERVICE; (B) THE SPECIFIC FEATURES OF THE SERVICE, OR ITS ACCURACY, RELIABILITY, AVAILABILITY, OR ABILITY TO MEET YOUR NEEDS; OR (C) THAT ANY CONTENT YOU SUBMIT WILL BE ACCESSIBLE ON THE SERVICE.
Limitation of Liability
EXCEPT AS REQUIRED BY APPLICABLE LAW, YOUTUBE, ITS AFFILIATES, OFFICERS, DIRECTORS, EMPLOYEES AND AGENTS WILL NOT BE RESPONSIBLE FOR ANY LOSS OF PROFITS, REVENUES, BUSINESS OPPORTUNITIES, GOODWILL, OR ANTICIPATED SAVINGS; LOSS OR CORRUPTION OF DATA; INDIRECT OR CONSEQUENTIAL LOSS; PUNITIVE DAMAGES CAUSED BY:
ERRORS, MISTAKES, OR INACCURACIES ON THE SERVICE;
PERSONAL INJURY OR PROPERTY DAMAGE RESULTING FROM YOUR USE OF THE SERVICE;
ANY UNAUTHORIZED USE OF THE SERVICE;
ANY INTERRUPTION OR CESSATION OF THE SERVICE;
ANY VIRUSES OR MALICIOUS CODE TRANSMITTED TO OR THROUGH THE SERVICE BY ANY THIRD PARTY;
ANY CONTENT WHETHER SUBMITTED BY A USER OR YOUTUBE, INCLUDING YOUR USE OF CONTENT; AND/OR
THE REMOVAL OR UNAVAILABILITY OF ANY CONTENT.
THIS PROVISION APPLIES TO ANY CLAIM, REGARDLESS OF WHETHER THE CLAIM ASSERTED IS BASED ON WARRANTY, CONTRACT, TORT, OR ANY OTHER LEGAL THEORY.
TO THE EXTENT PERMITTED BY APPLICABLE LAW, YOUTUBE AND ITS AFFILIATES’ TOTAL LIABILITY FOR ANY CLAIMS ARISING FROM OR RELATING TO THE SERVICE IS LIMITED TO THE GREATER OF: (A) THE AMOUNT OF REVENUE THAT YOUTUBE HAS PAID TO YOU FROM YOUR USE OF THE SERVICE IN THE 12 MONTHS BEFORE THE DATE OF YOUR NOTICE, IN WRITING TO YOUTUBE, OF THE CLAIM AND (B) USD $500, WHICHEVER IS HIGHER.
Indemnity
To the extent permitted by applicable law, you agree to defend, indemnify and hold harmless YouTube, its Affiliates, officers, directors, employees and agents, from and against any and all claims, damages, obligations, losses, liabilities, costs or debt, and expenses (including but not limited to attorney's fees) arising from: (i) your use of he Service; (ii) your violation of any term of these Terms of Service; (iii) your violation of any third party right, including without limitation any copyright, property, or privacy right; or (iv) any claim that your Content caused damage to a third party. This defense and indemnification obligation will survive this Agreement and your use of the Service.
Third-Party Links
The Service may contain links to third-party websites and online services that are not owned or controlled by YouTube. YouTube has no control over, and assumes no responsibility for, such websites and online services. Be aware when you leave the Service; we suggest you read the terms and privacy policy of each third-party website and online service that you visit.
About this Agreement
Changing this Agreement
We may change this Agreement, (1) to reflect changes to our Service or how we do business - for example, when we add new products or features or remove old ones, (2) for legal, regulatory, or security reasons, or (3) to prevent abuse or harm .
If we materially change this Agreement, we’ll provide you with reasonable advance notice and the opportunity to review the changes, except (1) when we launch a new product or feature, or (2) in urgent situations, such as preventing ongoing abuse or responding to legal requirements. If you don’t agree to the new terms, you should remove any Content you uploaded and stop using the Service.
Continuation of this Agreement
If your use of the Service ends, the following terms of this Agreement will continue to apply to you: “Other Legal Terms”, “About This Agreement”, and the licenses granted by you will continue as described under “Duration of License”.
Severance
If it turns out that a particular term of this Agreement is not enforceable for any reason, this will not affect any other terms.
No Waiver
If you fail to comply with this Agreement and we do not take immediate action, this does not mean that we are giving up any rights that we may have (such as the right to take action in the future).
Interpretation
In these terms, “include” or “including” means “including but not limited to,” and any examples we give are for illustrative purposes.
Governing Law
California law will govern all disputes arising out of or relating to these terms or the Service, regardless of conflict of laws rules. These disputes will be resolved exclusively in the federal or state courts of Santa Clara County, California, USA, and you and YouTube consent to personal jurisdiction in those courts.
If applicable local law prevents these disputes from being resolved in a California court, then you can file these disputes in your local courts. Likewise, if applicable local law prevents your local court from applying California law to resolve these disputes, then these disputes will be governed by the laws of your country of residence.




An "Account" represents your legal relationship with GitHub. A “Personal Account” represents an individual User’s authorization to log in to and use the Service and serves as a User’s identity on GitHub. “Organizations” are shared workspaces that may be associated with a single entity or with one or more Users where multiple Users can collaborate across many projects at once. A Personal Account can be a member of any number of Organizations.
The “Agreement” refers, collectively, to all the terms, conditions, notices contained or referenced in this document (the “Terms of Service” or the "Terms") and all other operating rules, policies (including the GitHub Privacy Statement, available at github.com/site/privacy) and procedures that we may publish from time to time on the Website. Most of our site policies are available at docs.github.com/categories/site-policy.
"Beta Previews" mean software, services, or features identified as alpha, beta, preview, early access, or evaluation, or words or phrases with similar meanings.
“Content” refers to content featured or displayed through the Website, including without limitation code, text, data, articles, images, photographs, graphics, software, applications, packages, designs, features, and other materials that are available on the Website or otherwise available through the Service. "Content" also includes Services. “User-Generated Content” is Content, written or otherwise, created or uploaded by our Users. "Your Content" is Content that you create or own.
“GitHub,” “We,” and “Us” refer to GitHub, Inc., as well as our affiliates, directors, subsidiaries, contractors, licensors, officers, agents, and employees.
The “Service” refers to the applications, software, products, and services provided by GitHub, including any Beta Previews.
“The User,” “You,” and “Your” refer to the individual person, company, or organization that has visited or is using the Website or Service; that accesses or uses any part of the Account; or that directs the use of the Account in the performance of its functions. A User must be at least 13 years of age. Special terms may apply for business or government Accounts (See Section B(5): Additional Terms).
The “Website” refers to GitHub’s website located at github.com, and all content, services, and products provided by GitHub at or through the Website. It also refers to GitHub-owned subdomains of github.com, such as education.github.com and pages.github.com. These Terms also govern GitHub’s conference websites, such as githubuniverse.com, and product websites, such as electronjs.org. Occasionally, websites owned by GitHub may provide different or additional terms of service. If those additional terms conflict with this Agreement, the more specific terms apply to the relevant page or service.


1. Account Controls
Users. Subject to these Terms, you retain ultimate administrative control over your Personal Account and the Content within it.

Organizations. The "owner" of an Organization that was created under these Terms has ultimate administrative control over that Organization and the Content within it. Within the Service, an owner can manage User access to the Organization’s data and projects. An Organization may have multiple owners, but there must be at least one Personal Account designated as an owner of an Organization. If you are the owner of an Organization under these Terms, we consider you responsible for the actions that are performed on or through that Organization.

2. Required Information
You must provide a valid email address in order to complete the signup process. Any other information requested, such as your real name, is optional, unless you are accepting these terms on behalf of a legal entity (in which case we need more information about the legal entity) or if you opt for a paid Account, in which case additional information will be necessary for billing purposes.

3. Account Requirements
We have a few simple rules for Personal Accounts on GitHub's Service.

You must be a human to create an Account. Accounts registered by "bots" or other automated methods are not permitted. We do permit machine accounts:
A machine account is an Account set up by an individual human who accepts the Terms on behalf of the Account, provides a valid email address, and is responsible for its actions. A machine account is used exclusively for performing automated tasks. Multiple users may direct the actions of a machine account, but the owner of the Account is ultimately responsible for the machine's actions. You may maintain no more than one free machine account in addition to your free Personal Account.
One person or legal entity may maintain no more than one free Account (if you choose to control a machine account as well, that's fine, but it can only be used for running a machine).
You must be age 13 or older. While we are thrilled to see brilliant young coders get excited by learning to program, we must comply with United States law. GitHub does not target our Service to children under 13, and we do not permit any Users under 13 on our Service. If we learn of any User under the age of 13, we will terminate that User’s Account immediately. If you are a resident of a country outside the United States, your country’s minimum age may be older; in such a case, you are responsible for complying with your country’s laws.
Your login may only be used by one person — i.e., a single login may not be shared by multiple people. A paid Organization may only provide access to as many Personal Accounts as your subscription allows.
You may not use GitHub in violation of export control or sanctions laws of the United States or any other applicable jurisdiction. You may not use GitHub if you are or are working on behalf of a Specially Designated National (SDN) or a person subject to similar blocking or denied party prohibitions administered by a U.S. government agency. GitHub may allow persons in certain sanctioned countries or territories to access certain GitHub services pursuant to U.S. government authorizations. For more information, please see our Export Controls policy.
4. Account Security
You are responsible for keeping your Account secure while you use our Service. We offer tools such as two-factor authentication to help you maintain your Account's security, but the content of your Account and its security are up to you.

You are responsible for all content posted and activity that occurs under your Account (even when content is posted by others who have Accounts under your Account).
You are responsible for maintaining the security of your Account and password. GitHub cannot and will not be liable for any loss or damage from your failure to comply with this security obligation.
You will promptly notify GitHub by contacting us through the GitHub Support portal if you become aware of any unauthorized use of, or access to, our Service through your Account, including any unauthorized use of your password or Account.
5. Additional Terms
In some situations, third parties' terms may apply to your use of GitHub. For example, you may be a member of an organization on GitHub with its own terms or license agreements; you may download an application that integrates with GitHub; or you may use GitHub to authenticate to another service. Please be aware that while these Terms are our full agreement with you, other parties' terms govern their relationships with you.

If you are a government User or otherwise accessing or using any GitHub Service in a government capacity, this Government Amendment to GitHub Terms of Service applies to you, and you agree to its provisions.

If you have signed up for GitHub Enterprise Cloud, the Enterprise Cloud Addendum applies to you, and you agree to its provisions.

Your use of the Website and Service must not violate any applicable laws, including copyright or trademark laws, export control or sanctions laws, or other laws in your jurisdiction. You are responsible for making sure that your use of the Service is in compliance with laws and any applicable regulations.

You agree that you will not under any circumstances violate our Acceptable Use Policies or Community Guidelines.

'''

In [12]:
prediction = summarize(test, BigBird_tokenizer)
prediction

'you can use this service for your personal use and for your advertising and marketing purposes. you can also use this for your business use and also for your own personal use. this service does not allow you to use it for advertising or marketing purposes and it does not guarantee that this service will not be used for you personal use or for any other business use. it does guarantee that it will not violate the privacy of the users of this service if it is used for advertising purposes or for other purposes. it also guarantees that it is not used for creating a false impression of the service or for making false claims about the service if the user does not agree with the service and the user has the right to use the service for marketing purposes or even for creating false claims regarding the service. the service may not use the user s personal use for the purposes of advertising or promotional purposes and the use of any other service in violation of the user privacy guarantee or 